<br><p style="font-family: Arial; font-size:3.75em;color:purple; font-style:bold">
Pandas <br> <br> <br>pivot tables</p><br><br>

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

We are going to use Titanic data for this part.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
titanic=sns.load_dataset('titanic')

In [ ]:
#Check the data
titanic.head()
titanic.describe().transpose()

The survival rate of men and women.

In [ ]:
titanic.groupby('sex')[['survived']].mean()

Two-dimensional Groupby will be made easier with pivot_table.

In [ ]:
#Survival rate of different genders in different cabin classes.
titanic.pivot_table('survived',index='sex',columns='class')

In [ ]:
#Using groupby()
titanic.groupby(['sex','class'])['survived'].mean().unstack()

To make pivot table with a continuous variable. We bin the variable first.

In [ ]:
#Survival rate in different age groups
age=pd.cut(titanic['age'],[0,18,80])
age.dropna()
age.unique()
titanic.pivot_table('survived',['sex',age],columns='class')

In [ ]:
#Survival rate in different fare groups
fare=pd.qcut(titanic['fare'],2)
titanic.pivot_table('survived',['sex',age],[fare,'class'])

Options of pivot_table: 1. Specifying different aggregation function (by default is mean).

In [ ]:
titanic.pivot_table(index='sex',columns='class',aggfunc={'survived':'sum','fare':'mean'})

2. To compute totals along dimensions

In [ ]:
titanic.pivot_table('survived','sex','class',margins=True)

<br><p style="font-family: Arial; font-size:3.75em;color:purple; font-style:bold">
Pandas string operations <br> <br> <br>Recipe database</p><br><br>

In [ ]:
!curl -O http://openrecipes.s3.amazonaws.com/recipeitems-latest.json.gz
#!gunzip recipeitems-latest.json.gz

In [ ]:
from sh import gunzip
gunzip('recipeitems-latest.json.gz')

The try statement works as follows.

<p>First, the try clause (the statement(s) between the try and except keywords) is executed.</p>
<p>If no exception occurs, the except clause is skipped and execution of the try statement is finished.</p>
<p>If an exception occurs during execution of the try clause, the rest of the clause is skipped. Then if its type matches the exception named after the except keyword, the except clause is executed, and then execution continues after the try statement.</p>
</p>If an exception occurs which does not match the exception named in the except clause, it is passed on to outer try statements; if no handler is found, it is an unhandled exception and execution stops with a message as shown above.</p>

In [ ]:
try:
    recipes = pd.read_json('recipeitems-latest.json')
except ValueError as e:
    print("ValueError:", e)

In [ ]:
#import gzip
with open('openrecipes.json','r') as fin:
    data = (line.strip() for line in fin)
    data_json = '[{0}]'.format(','.join(data))
recipes = pd.read_json(data_json)

In [ ]:
recipes.shape
recipes.iloc[0]
recipes.head()

In [ ]:
recipes.ingredients.str.len().describe()

In [ ]:
recipes.name[np.argmax(recipes.ingredients.str.len())]

In [ ]:
#How many breakfast recipes
recipes.description.str.contains('[Bb]reakfast').sum()

In [ ]:
#How many list cinnamon as an ingredient
recipes.ingredients.str.contains('[Cc]innamon').sum()

In [ ]:
#Given a list of ingredients, find a recipe that uses all these ingredients
#We'll start with a list of common ingredients

spice_list = ['salt', 'pepper', 'oregano', 'sage', 'parsley',
              'rosemary', 'tarragon', 'thyme', 'paprika', 'cumin']

In [ ]:
import re
spice_df = pd.DataFrame(dict((spice, recipes.ingredients.str.contains(spice, re.IGNORECASE))
                             for spice in spice_list)) #a dictionary of Series objects
spice_df.head()

In [ ]:
#Find a recipe uses oregano and parsley
selection = spice_df.query('oregano & parsley')
recipes.name[selection.index]
recipes.iloc[selection.index]

In [ ]:
for idx in selection.index:
    print(idx)
    print(recipes.iloc[idx])

In [ ]:
recipes.iloc[selection.index]

<br><p style="font-family: Arial; font-size:3.75em;color:purple; font-style:bold">
Matplotlib: Exploring <br> <br> <br>Data Visualization</p><br><br>

<p>Data Source: https://www.kaggle.com/worldbank/world-development-indicators</p>
The World Development Indicators dataset obtained from the World Bank contains over a thousand annual indicators of economic development from hundreds of countries around the world. 

# Step 1: Initial exploration of the Dataset

In [ ]:
import matplotlib.pyplot as plt
import random

In [ ]:
#Import data
world=pd.read_csv('Indicators.csv')
world.shape

This is a really large dataset, at least in terms of the number of rows.

In [ ]:
world.head()

Looks like it has different indicators for different countries with the year and value of the indicator. 

### How many UNIQUE country names are there ?

In [ ]:
countries=world['CountryName'].unique()
len(countries)

### Are there same number of country codes ?

In [ ]:
countcodes=world['CountryCode'].unique()
len(countcodes)

### Are there many indicators or few ?

In [ ]:
inds=world['IndicatorName'].unique()
len(inds)

In [ ]:
#check for one year
ind2000=world[world['Year']==2000]['IndicatorName'].unique()
len(ind2000)

In [ ]:
indUSA2000 = world[(world['Year']==2000) & (world['CountryName']=='United States')]['IndicatorName'].unique()
len(indUSA2000)

### How many years of data do we have ?

In [ ]:
# How many years of data do we have ?
years=world['Year'].unique()
len(years)

### What's the range of years?

In [ ]:
print(min(years), 'to',max(years))

<p style="font-family: Arial; font-size:2.5em;color:blue; font-style:bold">
Matplotlib: Basic Plotting, Part 1</p><br>

### Lets pick a country and an indicator to explore: CO2 Emissions per capita and the USA

In [ ]:
# select CO2 emissions for the United States
hist_indicator='CO2 emissions \(metric'#Regular expression operations
hist_country='USA'

mask1=world['IndicatorName'].str.contains(hist_indicator)
mask2=world['CountryCode'].str.contains(hist_country)

USAco2=world[mask1 & mask2]

In [ ]:
USAco2.head()

### Let's see how emissions have changed over time using MatplotLib

In [ ]:
# get the years
years=USAco2['Year'].values
# get the values 
co2=USAco2['Value'].values

# create
plt.bar(years,co2)
#If coding in a script, you may use plt.show() to display plots.
#It is suggested that plt.show() should be used only once per Python session, usually the very end of the script.
plt.show()

Turns out emissions per capita have dropped a bit over time, but let's make this graphic a bit more appealing before we continue to explore it.

In [ ]:
#This works for Notebook
%matplotlib inline
# switch to a line plot
plt.plot(years,co2)
# Label the axes
plt.xlabel('Year')
plt.ylabel(USAco2['IndicatorName'].iloc[0])
#label the figure
plt.title('CO2 Emissions in USA')

plt.axis([1959,2011,0,25])

### Using Histograms to explore the distribution of values
We could also visualize this data as a histogram to better explore the ranges of values in CO2 production per year. 

In [ ]:
#To plot 'Value'
fig=plt.figure()
plt.hist(co2,10,facecolor='green')

plt.xlabel(USAco2['IndicatorName'].iloc[0])
plt.ylabel('# of Years')
plt.title('CO2 Histogram')

plt.grid(True)

In [ ]:
# the histogram of the data


In [ ]:
#save the figure
fig.savefig('co2hist.jpeg')
fig.canvas.get_supported_filetypes()

### But how do the USA's numbers relate to those of other countries?

In [ ]:
# select CO2 emissions for all countries in 2011
hist_indicator='CO2 emissions \(metric'
hist_year=2011

mask1 = world['IndicatorName'].str.contains(hist_indicator)
mask2=world['Year'].isin([hist_year])

# apply our mask
co2_2011 = world[mask1 & mask2]
co2_2011.head()

For how many countries do we have CO2 per capita emissions data in 2011

In [ ]:
print(len(co2_2011))

In [ ]:
# let's plot a histogram of the emmissions per capita by country
#MATLAB interface
plt.hist(co2_2011['Value'],10,facecolor='green')

plt.xlabel(co2_2011['IndicatorName'].iloc[0])
plt.ylabel('# of Countries')
plt.title('CO2 histogram over countries')

plt.annotate('USA',xy=(18,5),xycoords='data',
            xytext=(18,30),textcoords='data',
            arrowprops=dict(arrowstyle='->',
                           connectionstyle='arc3'))


In [ ]:
#Object-oriented interface
fig,ax=plt.subplots()

ax.hist(co2_2011['Value'],10,facecolor='green')

ax.set_xlabel(co2_2011['IndicatorName'].iloc[0])
ax.set_ylabel('# of Countries')
ax.set_title('CO2 histogram over countries')

ax.annotate('USA',xy=(18,5),xycoords='data',
            xytext=(18,30),textcoords='data',
            arrowprops=dict(arrowstyle='->',
                           connectionstyle='arc3'))
ax.grid(True)

In [ ]:
#axes.set
fig,ax=plt.subplots()

ax.hist(co2_2011['Value'],10,facecolor='green')

ax.set(xlabel=(co2_2011['IndicatorName'].iloc[0]),
       ylabel='# of Countries',
       title='CO2 histogram over countries')

ax.annotate('USA',xy=(18,5),xycoords='data',
            xytext=(18,30),textcoords='data',
            arrowprops=dict(arrowstyle='->',
                           connectionstyle='arc3'))
ax.grid(True)

So the USA, at ~18 CO2 emissions (metric tons per capital) is quite high among all countries.

### Relationship between GDP and CO2 Emissions in USA

In [ ]:
# select GDP Per capita emissions for the United States
hist_indicator='GDP per capita \(constant 2005'
hist_country='USA'

mask1=world['IndicatorName'].str.contains(hist_indicator)
mask2=world['CountryCode'].str.contains(hist_country)

USAgdp=world[mask1 & mask2]

#plot

In [ ]:
USAgdp.head(2)

In [ ]:
USAco2.head(2)

In [ ]:
# switch to a line plot
gdp=USAgdp['Value'].values
years=USAgdp['Year'].values
plt.plot(years,gdp)
# Label the axes


#label the figure


# to make more honest, start the y axis at 0
plt.axis([1959, 2011,0,50000])

So although we've seen a decline in the CO2 emissions per capita, it does not seem to translate to a decline in GDP per capita

### ScatterPlot for comparing GDP against CO2 emissions (per capita)

First, we'll need to make sure we're looking at the same time frames

In [ ]:
#Do the two indicators match?
print("GDP Min Year:",USAgdp['Year'].min(), "Max", USAgdp['Year'].max())
print("Co2 Min Year:",USAco2['Year'].min(), "Max", USAco2['Year'].max())

We have 3 extra years of GDP data, so let's trim those off so the scatterplot has equal length arrays to compare (this is actually required by scatterplot)

In [ ]:
USAgdp_trunc=USAgdp[USAgdp['Year']<2012]
len(USAgdp_trunc)
len(USAco2)

In [ ]:
fig,ax=plt.subplots()

X=USAgdp_trunc['Value']
Y=USAco2['Value']
ax.scatter(X,Y)

In [ ]:
#Use plt.plot()
#for types of symbols, check https://matplotlib.org/api/markers_api.html

This doesn't look like a strong relationship.  We can test this by looking at correlation.

In [ ]:
np.corrcoef(USAgdp_trunc['Value'],USAco2['Value'])

A correlation of 0.07 is pretty weak.

You could continue to explore this to see if other countries have a closer relationship between CO2 emissions and GDP.  Perhaps it is stronger for developing countries?

In [ ]:
#Excercise: find the relationship between GDP and CO2 emission for a developing country


let's see if we can find some more indicators in common between countries.  

In [ ]:
# Filter 1

# Picks years of choice
yearfilter=[2010,2011,2012,2013,2014]

In [ ]:
# Filter 2 

# Pick 2 countries randomly
countries = data['CountryName'].unique().tolist()
indicators = data['IndicatorName'].unique().tolist()

countryfilter=random.sample(countries,2)
countryfilter

In [ ]:
# Filter 3

# Pick 1 Indicator randomly
indicatorfilter=random.sample(indicators,1)
indicatorfilter

# Problem:  We're missing data.
# Not all countries have all indicators for all years

To solve this, we'll need to find two countries and two indicators for which we have data over this time range.

In [ ]:
#retriving data for Country 1
filtermesh=(data['CountryName']==countryfilter[0])&(data['IndicatorName'].isin(indicatorfilter))&(data['Year'].isin(yearfilter))
country1=data.loc[filtermesh]
country1

In [ ]:
#retriving data for Country 2
filtermesh=(data['CountryName']==countryfilter[1])&(data['IndicatorName'].isin(indicatorfilter))&(data['Year'].isin(yearfilter))
country2=data.loc[filtermesh]
country2

# So let's pick indicators and countries which have data over this time range

The code below will randomly pick countries and indicators until it finds two countries who have data for an indicator over this time frame.

In [ ]:

#countryFilter:    pick two countries, 
#indicatorsFilter: pick an indicator, 
#yearsFilter: plot for years in yearsFilter

# problem - not all countries have all indicators so if you go to visualize, it'll have missing data.
# randomly picking two indicators and countries, do these countries have valid data over those years.
# brings up the discussion of missing data/ missing fields
# until we find full data

filterdata1=[]
filterdata2=[]

while (len(filterdata1)<len(yearfilter)):
    #pick new indicators and countries
    indicatorfilter=random.sample(indicators,1)
    countryfilter=random.sample(countries,2)
    #retrieve country 1 data
    filtermesh=(data['CountryName']==countryfilter[0])&(data['IndicatorName'].isin(indicatorfilter))&(data['Year'].isin(yearfilter))
    filterdata1=data.loc[filtermesh]
    
    #print out when while conidition is true
    if (len(filterdata1)<len(yearfilter)):
        print('Keep looking')



In [ ]:
filterdata1

In [ ]:
#Country 2
while (len(filterdata2)<len(filterdata1)):
    filtermesh=(data['CountryName']==countryfilter[1])&(data['IndicatorName'].isin(indicatorfilter))&(data['Year'].isin(yearfilter))
    filterdata2=data.loc[filtermesh]
    #pick new countries
    countryfilter[1]=random.sample(countries,1)[0]
    
    if (len(filterdata2)<len(filterdata1)):
        print('Keep looking')

In [ ]:
#Get the intersection of the two
if len(filterdata1)<len(filterdata2):
    small=len(filterdata1)
else:
    small=len(filterdata2)

In [ ]:
filterdata1=filterdata1[0:small]
filterdata2=filterdata2[0:small]

In [ ]:
#To plot it
%matplotlib inline
fig=plt.figure()
ax=plt.axes()

ax.yaxis.grid(True)
ax.set_title(indicatorfilter[0], fontsize=18)
ax.set_xlabel(filterdata1['CountryName'].iloc[0], fontsize=10)
ax.set_ylabel(filterdata2['CountryName'].iloc[0], fontsize=10)

X=filterdata1['Value']
Y=filterdata2['Value']

ax.scatter(X,Y)

## Want more ? 

### Matplotlib Examples Library

http://matplotlib.org/examples/index.html